In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import corner

from cosmic.evolve import Evolve
from cosmic.sample.initialbinarytable import InitialBinaryTable


In [ ]:
#load your data here, you'll probably have a different filename
dat = np.load('WDMS.npz')

In [ ]:
# select out the burn in (in this case, 300 steps)
dat = dat['chain'][0:1024,300:,:]

In [ ]:
n_chain_list = len(dat)
n_chains = len(dat[0])
n_params = len(dat[0][0])

#create an array and downsample by a factor of 5 ('thin the chains by a factor of 5')
dat_stack = dat.reshape(n_chain_list * n_chains, n_params)

dat_stack = dat_stack[::5,:]

In [ ]:
# save as a dataframe since these are easier to read (at least for kb)
param_list = ['m1', 'm2', 'logtb', 'ecc', 'alpha_1', 'lambda']
dat_stack = pd.DataFrame(dat_stack, columns=param_list)


In [ ]:
corner.corner(dat_stack[['m1', 'm2', 'logtb', 'ecc', 'alpha_1', 'lambda']], labels=['m1', 'm2', 'logtb', 'ecc', 'alpha_1', 'lambda']);

### Now we can re-evolve the binaries to see what their evolutionary pathways are given the initial conditions and sampling parameters. This is pretty hacky since we need to have an initial binary table that has all of the BSEDict flags populated. To make that happen, we first evolve a population, then overwrite the initial conditions and re-run

In [ ]:
BSEDict = {'xi': 1.0, 'bhflag': 1, 'neta': 0.5, 'windflag': 3, 'wdflag': 1, 'alpha1': [1.0, 1.0], 'pts1': 0.01, 
           'pts3': 0.02, 'pts2': 0.01, 'epsnov': 0.001, 'hewind': 0.5, 'ck': 1000, 'bwind': 0.0, 
           'lambdaf': 0.0, 'mxns': 3.0, 'beta': -1.0, 'tflag': 1, 'acc2': 1.5, 'grflag' : 1, 'remnantflag': 4, 
           'ceflag': 1, 'eddfac': 1.0, 'ifflag': 0, 'bconst': 3000, 'sigma': 265, 'gamma': -2.0, 'pisn': -2, 
           'natal_kick_array' : [[-100.0,-100.0,-100.0,-100.0,0.0], [-100.0,-100.0,-100.0,-100.0,0.0]], 
           'bhsigmafrac' : 1.0, 'polar_kick_angle' : 90, 
           'qcrit_array' : [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0], 
           'cekickflag' : 2, 'cehestarflag' : 0, 'cemergeflag' : 1, 'ecsn' : 2.25, 'ecsn_mlow' : 1.6, 'aic' : 1, 
           'ussn' : 1, 'sigmadiv' :-20.0, 'qcflag' : 5, 'eddlimflag' : 0, 
           'fprimc_array' : [2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,
                             2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0], 
           'bhspinflag' : 0, 'bhspinmag' : 0.0, 'rejuv_fac' : 1.0, 'rejuvflag' : 0, 'htpmb' : 1, 'ST_cr' : 1, 'ST_tide' : 1, 
           'bdecayfac' : 1, 'rembar_massloss' : 0.5, 'kickflag' : 0, 'zsun' : 0.014, 'bhms_coll_flag' : 0, 
           'don_lim' : -1, 'acc_lim' : [-1, -1], 'rtmsflag':0}

In [ ]:
InitialBinaries, mass_singles, mass_binaries, n_singles, n_binaries = InitialBinaryTable.sampler(
    'independent', [10,11,12], [0,1], binfrac_model=1.0, primary_model='kroupa01', 
    ecc_model='uniform', porb_model='raghavan10', m2_min=0.08, SF_start=13700.0, SF_duration=0.0, 
    met=0.316*0.02, size=1000)

In [ ]:
bpp, bcm, initC, kick_info  = Evolve.evolve(initialbinarytable=InitialBinaries, BSEDict=BSEDict)
initC = initC.sample(1000, replace=False)

In [ ]:
dat_in = dat_stack.sample(1000, replace=False)

In [ ]:
dat_in

In [ ]:
# you might need to update the initC columns based on what we decide to sample over. 
initC['mass_1'] = dat_in.m1.values
initC['mass_2'] = dat_in.m2.values
initC['mass0_1'] = dat_in.m1.values
initC['mass0_2'] = dat_in.m2.values
initC['ecc'] = dat_in.ecc.values
initC['porb'] = 10**dat_in.logtb.values
#lambda means something in python
initC['lambdaf'] = dat_in['lambda'].values
initC['alpha1_1'] = dat_in.alpha_1.values


### This time, we don't specify the BSEDict since it's already specified

In [ ]:
bpp_re, bcm_re, initC_re, kick_info_re  = Evolve.evolve(initialbinarytable=initC, BSEDict={})

In [ ]:
bpp_re